In [2]:
# main reference
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

import random
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

SOS_token = 0
EOS_token = 1

raw

['I feel hungry.\t나는 배가 고프다.',
 'Pytorch is very easy.\t파이토치는 매우 쉽다.',
 'Pytorch is a framework for deep learning.\t파이토치는 딥러닝을 위한 프레임워크이다.',
 'Pytorch is very clear to use.\t파이토치는 사용하기 매우 직관적이다.']

#### 단어사전 Class

In [3]:
class Vocab:
    def __init__(self):                                                                                             # 'hello seq to seq model '
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}     # {'<SOS>': 0, '<EOS>': 1, 'hello': 2, 'seq': 3, 'to': 4, 'model': 5, '': 6}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}     # {0: '<SOS>', 1: '<EOS>', 2: 'hello', 3: 'seq', 4: 'to', 5: 'model', 6: ''}
        self.vocab_count = {}                                                                                  # {'hello': 1, 'seq': 2, 'to': 1, 'model': 1, '': 1}
        self.n_vocab = len(self.vocab2index)                                                       # 7

    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1

##### 단어사전 test

In [4]:
test_sentence = 'hello seq to seq model '

test_vocab = Vocab()
test_vocab.add_vocab(test_sentence)
print("vocab2index: \t", test_vocab.vocab2index)
print("index2vocab: \t", test_vocab.index2vocab)
print("vocab_count: \t", test_vocab.vocab_count)
print("n_vocab: \t\t", test_vocab.n_vocab)

vocab2index: 	 {'<SOS>': 0, '<EOS>': 1, 'hello': 2, 'seq': 3, 'to': 4, 'model': 5, '': 6}
index2vocab: 	 {0: '<SOS>', 1: '<EOS>', 2: 'hello', 3: 'seq', 4: 'to', 5: 'model', 6: ''}
vocab_count: 	 {'hello': 1, 'seq': 2, 'to': 1, 'model': 1, '': 1}
n_vocab: 		 7


#### filter_pair : pair(src, tgt)가 모두 Max Length 이하인지 여부

In [5]:
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [6]:
pairs = []
for line in raw:
    pairs.append([s for s in line.strip().lower().split("\t")])
print("Read {} sentence pairs".format(len(pairs)))
print(pairs[0])

print(filter_pair(pairs[0], 10, 12))
print(filter_pair(pairs[0], 3, 3))

Read 4 sentence pairs
['i feel hungry.', '나는 배가 고프다.']
True
False


### 전처리 

In [7]:
def preprocess(corpus, source_max_length, target_max_length):

    # 문장별로 pair 간단히 만들어줌
    print("[preprocess] reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("[preprocess] Read {} sentence pairs".format(len(pairs)))

    # (src, tgt) 둘다 max length 이하인 경우만 통과
    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("[preprocess] Trimmed to {} sentence pairs".format(len(pairs)))

    # vocab 2개 생성
    source_vocab = Vocab() 
    target_vocab = Vocab()

    # src들 → src_vocab에 추가, tgt들 → tgt_vocab에 추가
    print("[preprocess] Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("[preprocess] source vocab size =", source_vocab.n_vocab)
    print("[preprocess] target vocab size =", target_vocab.n_vocab)

    # 리턴, pairs원본, 소스모음vocab, 타겟모음vocab
    return pairs, source_vocab, target_vocab

In [8]:
preprocess(raw, 10, 12)

[preprocess] reading corpus...
[preprocess] Read 4 sentence pairs
[preprocess] Trimmed to 4 sentence pairs
[preprocess] Counting words...
[preprocess] source vocab size = 17
[preprocess] target vocab size = 13


([['i feel hungry.', '나는 배가 고프다.'],
  ['pytorch is very easy.', '파이토치는 매우 쉽다.'],
  ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'],
  ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']],
 <__main__.Vocab at 0x7fdbea77ba20>)

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
#         print("self.embedding(x).shape:", self.embedding(x).shape)
#         print("self.embedding(x).view(1, 1, -1).shape:", self.embedding(x).view(1, 1, -1).shape)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [10]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))                        # 3D → 2D  # tensor([[[-0.1669, ... ,  0.4492]]])  → tensor([[-0.1669, ... ,  0.4492]]) ※ 요소 : 16개(hidden_size)
        return x, hidden

#### 토큰화

In [26]:
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1) # 특성1개 모양으로 : [2, 3, 4, 3, 5, 6, 1] → [ [2], [3], [4], [3], [5], [6], [1] ] 

##### 토큰화 테스트

In [28]:
print(test_sentence)
print(test_vocab.index2vocab)
tensorize(test_vocab, test_sentence)

hello seq to seq model 
{0: '<SOS>', 1: '<EOS>', 2: 'hello', 3: 'seq', 4: 'to', 5: 'model', 6: ''}


tensor([[2],
        [3],
        [4],
        [3],
        [5],
        [6],
        [1]], device='cuda:0')

##### n_iter만큼 랜덤 묶음 batch만큼 Pair 구성 테스트

In [13]:
[random.choice(pairs) for _ in range(10)]

[['pytorch is very easy.', '파이토치는 매우 쉽다.'],
 ['i feel hungry.', '나는 배가 고프다.'],
 ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.'],
 ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'],
 ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'],
 ['pytorch is very easy.', '파이토치는 매우 쉽다.'],
 ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'],
 ['i feel hungry.', '나는 배가 고프다.'],
 ['pytorch is very easy.', '파이토치는 매우 쉽다.'],
 ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.']]

### Train

In [14]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0

    # opt, loss  설정
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    
    # 배치단위로 src, tgt를 토큰화
    training_batch = [random.choice(pairs) for _ in range(n_iter)] # n_iter만큼 랜덤묶음 batch 구성해줌
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    # batch 단위로 보면 될 듯(?)
    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1] # idx를 1부터 시작해서 1씩 빼준 것 뿐
        target_tensor = training_target[i - 1]   
        
        # 최초 hidden은 0으로 채워진 모양으로 준비
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)     

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0
        
        # source문장 돌면서 토큰단위로 Encoding
        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)    # 결과 hidden 예 :  tensor([[[-0.2580, ... , -0.1358]]]  ※히든 갯수 만큼

        # Decoding 준비
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden

        # target문장 돌면서 Decoder Loss →  훈련(back, step)
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing
            
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

In [15]:
#  # RuntimeError: legacy constructor expects device type: cpubut device type: cuda was passed
#  torch.Tensor([0], device=device)

torch.Tensor([0]).to(device)

tensor([0.], device='cuda:0')

### 평가

In [16]:
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])
        print("=", pair[1])

        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        
        # Encoding 준비
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        # source문장 돌면서 Encoding     
        for ei in range(source_length):       
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)     # 결과 hidden 예 :  tensor([[[-0.2580, ... , -0.1358]]]  ※히든 갯수 만큼

        # Decoding 준비
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)                   
        decoder_hidden = encoder_hidden                                                      
        
        # Decoding
        decoded_words = []
        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            
            # 디코더 끝이면(<EOS>) 종료
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")

### main

In [17]:
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print("\n▶Pair모양 확인(random) : ", random.choice(load_pairs))

print("\n▶인코더/디코더 선언")
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

print("\n▶훈련시작")
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 100, print_every=1000)

print("\n▶전체 평가")
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

[preprocess] reading corpus...
[preprocess] Read 4 sentence pairs
[preprocess] Trimmed to 4 sentence pairs
[preprocess] Counting words...
[preprocess] source vocab size = 17
[preprocess] target vocab size = 13

▶Pair모양 확인(random) :  ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']

▶인코더/디코더 선언

▶훈련시작

▶전체 평가
> i feel hungry.
= 나는 배가 고프다.
< 파이토치는 매우 <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 매우 <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 매우 <EOS>

